In [ ]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT, force_remount=True)           # we mount the google drive at /content/drive

/content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%cd 'drive/My Drive/Workspaces/machinelearning/Projects/Udacity MLND/Capstone Project/'
# %pwd

/content/drive/My Drive/Workspaces/machinelearning/Projects/Udacity MLND/Capstone Project


In [ ]:
!pip install textstat
!pip install pyLDAvis

In [ ]:
# Imports
import string
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from statistics import *
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import textstat

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

import pandas as pd
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

import numpy as np

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import folium 
from folium import plugins 

import re


In [ ]:

#utility functions:
def plot_readability(a,b,title,bins=0.1,colors=['#3A4750', '#F64E8B']):
  trace1 = ff.create_distplot([a,b], [" Real disaster tweets","Not real disaster tweets"], bin_size=bins, colors=colors, show_rug=False)
  
  trace1['layout'].update(title=title)
  py.iplot(trace1, filename='Distplot')
  table_data= [["Statistical Measures"," Not real disaster tweets","real disaster tweets"], ["Mean",mean(a),mean(b)], ["Standard Deviation",pstdev(a),pstdev(b)],
               ["Variance",pvariance(a),pvariance(b)],
               ["Median",median(a),median(b)],
               ["Maximum value",max(a),max(b)],
               ["Minimum value",min(a),min(b)]]
  trace2 = ff.create_table(table_data)
  py.iplot(trace2, filename='Table')

punctuations = string.punctuation
stopwords = list(STOP_WORDS)

parser = English()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens


def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

def removeurl(raw_text):
  clean_text = re.sub(r'^https?:\/\/.*[\r\n]*', '', raw_text, flags=re.MULTILINE)
  return clean_text

### Import Dataset


In [ ]:
# Importing cleaned data
train_df = pd.read_csv('data/preprocessed_train.csv')
test_df = pd.read_csv('data/preprocessed_test.csv')
sub_df = pd.read_csv('data/sample_submission.csv')


In [ ]:
train_df.head()

,Unnamed: 0,id,keyword,location,text,target,word_count,unique_word_count,stopword_count,url_count,mean_word_length,char_count,punctuation_count,hashtag_count,mention_count,text_cleaned,target_relabeled
0,0,1,no_keyword,no_location,Our Deeds are the Reason of this #earthquake M...,1,13,13,5,0,4.384615,69,1,1,0,Our Deeds are the Reason of this # earthquake...,1
1,1,4,no_keyword,no_location,Forest fire near La Ronge Sask. Canada,1,7,7,0,0,4.571429,38,1,0,0,Forest fire near La Ronge Sask . Canada,1
2,2,5,no_keyword,no_location,All residents asked to 'shelter in place' are ...,1,22,20,9,0,5.090909,133,3,0,0,All residents asked to ' shelter in place ' ...,1
3,3,6,no_keyword,no_location,"13,000 people receive #wildfires evacuation or...",1,8,8,1,0,7.125000,65,2,1,0,"13,000 people receive # wildfires evacuation ...",1
4,4,7,no_keyword,no_location,Just got sent this photo from Ruby #Alaska as ...,1,16,15,6,0,4.500000,88,2,2,0,Just got sent this photo from Ruby # Alaska a...,1


### Topic Modeling

In [ ]:
notreal_text = train_df[train_df["target_relabeled"] == 0]['text_cleaned'].apply(spacy_tokenizer)
real_text = train_df[train_df["target_relabeled"] == 1]['text_cleaned'].apply(spacy_tokenizer)
#count vectorization
vectorizer_notreal = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
notreal_vectorized = vectorizer_notreal.fit_transform(notreal_text)
vectorizer_real = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
real_vectorized = vectorizer_real.fit_transform(real_text)

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:6: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:6: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:6: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:6: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid escape sequence \-

<input>:6: DeprecationWarning:

invalid escape sequence \-

<input>:4: DeprecationWarning:

invalid 

#### Latent Dirichlet Allocation (LDA)

Latent Dirichlet Allocation (LDA) and LSA are based on the same underlying assumptions: the distributional hypothesis, (i.e. similar topics make use of similar words) and the statistical mixture hypothesis (i.e. documents talk about several topics) for which a statistical distribution can be determined. The purpose of LDA is mapping each document in our corpus to a set of topics which covers a good deal of the words in the document.

What LDA does in order to map the documents to a list of topics is assign topics to arrangements of words, e.g. n-grams such as best player for a topic related to sports. This stems from the assumption that documents are written with arrangements of words and that those arrangements determine topics.LDA ignores syntactic information and treats documents as bags of words. It also assumes that all words in the document can be assigned a probability of belonging to a topic. That said, the goal of LDA is to determine the mixture of topics that a document contains.

In other words, LDA assumes that topics and documents look like this:

In [ ]:
# Latent Dirichlet Allocation Model
lda_notreal = LatentDirichletAllocation(n_components=10, max_iter=5, learning_method='online',verbose=True)
notreal_lda = lda_notreal.fit_transform(notreal_vectorized)
lda_real = LatentDirichletAllocation(n_components=10, max_iter=5, learning_method='online',verbose=True)
real_lda = lda_real.fit_transform(real_vectorized)

iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5
iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5


In [ ]:
def selected_topics(model, vectorizer, top_n=10):
  for idx, topic in enumerate(model.components_):
    print("Topic %d:" % (idx))
    print([(vectorizer.get_feature_names()[i], topic[i])
    for i in topic.argsort()[:-top_n - 1:-1]]) 

In [ ]:
print("Not real disaster tweets LDA Model:")
selected_topics(lda_notreal, vectorizer_notreal)

Not real disaster tweets LDA Model:
Topic 0:
[('like', 258.35115288572183), ('feel', 44.02364997980254), ('screamed', 37.740375363361956), ('movie', 33.83106365298814), ('crash', 33.11515197334516), ('sinking', 32.47019923389898), ('police', 31.44216342128816), ('traumatised', 30.037588371858263), ('phone', 29.327434536015463), ('bag', 28.096364319072734)]
Topic 1:
[('time', 86.60435533623885), ('know', 64.1433493489033), ('wreck', 63.995610822167016), ('best', 53.52205133486489), ('look', 43.18808357441017), ('shit', 34.01559531867751), ('need', 33.298513970481004), ('little', 31.606884506915083), ('wanna', 31.222028646301634), ('nuclear', 31.144642622756823)]
Topic 2:
[('loud', 127.50503862945044), ('bang', 123.5713652694648), ('heard', 116.96413574392392), ('ass', 54.90162552683071), ('smoke', 43.47649963825058), ('burning', 43.209056014505194), ('coming', 33.07769700254367), ('song', 31.118650270381544), ('hope', 30.20999844344853), ('girl', 29.05021180237064)]
Topic 3:
[('news', 6

In [ ]:
print("Real disaster tweets LDA Model:")
selected_topics(lda_real, vectorizer_real)

Real disaster tweets LDA Model:
Topic 0:
[('malaysia', 226.32141294237573), ('flight', 153.28990775748957), ('airlines', 145.50096085875356), ('wreckage', 93.5933607187942), ('investigators', 72.1266027866486), ('conclusively', 68.03927159076456), ('fatal', 55.92629981838036), ('earthquake', 50.42694430419583), ('reunion', 41.11734238838591), ('debris', 40.356275525222635)]
Topic 1:
[('news', 83.3340746284871), ('old', 74.15312831106962), ('emergency', 66.41840010319916), ('dead', 64.99229203326243), ('migrants', 45.39469278592565), ('kills', 40.28590192701572), ('hundreds', 35.614325284244345), ('day', 32.052394018965124), ('boat', 29.644960579506826), ('bestnaijamade', 27.195990317332743)]
Topic 2:
[('typhoon', 73.94567587937361), ('obama', 62.94893277801283), ('severe', 58.04635553566578), ('warning', 54.482309448980104), ('water', 53.993769710011165), ('war', 36.28178588982243), ('released', 35.25663432577489), ('live', 34.059169418242924), ('devastated', 33.831597023272224), ('mur

##### Visualizing LDA results of not a real disaster tweets with pyLDAvis

In [ ]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda_notreal, notreal_vectorized, vectorizer_notreal, mds='tsne')
dash

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
7      32.420952   8.115230       1        1  10.845645
2       0.648955  71.707100       2        1  10.735420
9      71.019684   9.971006       3        1  10.630399
1       3.785981  33.185135       4        1  10.352581
6     -42.666500  -9.222070       5        1  10.320552
3      -3.621689 -43.526516       6        1   9.986428
8      -3.632402  -4.171708       7        1   9.676016
4     -36.584400  35.804413       8        1   9.335174
0      44.391380  50.615383       9        1   9.060009
5      40.235809 -31.591665      10        1   9.057777, topic_info=             Term        Freq       Total Category  logprob  loglift
720          like  232.000000  232.000000  Default  30.0000  30.0000
529         great  134.000000  134.000000  Default  29.0000  29.0000
752          loud  117.000000  117.000000  Default  28.0000  28.0000
81           bang  114.000000  114.000000  Default  27.0000  27.0000
842           new  158.000000  158.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
842           new  112.640993  158.262292  Topic10  -2.7762   2.0615
1169    stretcher   22.656018   31.387540  Topic10  -4.3800   2.0756
1268  traumatised   23.449408   39.181397  Topic10  -4.3456   1.8882
113         blast   13.614079   16.495261  Topic10  -4.8893   2.2096
1354    whirlwind   12.572368   30.390200  Topic10  -4.9689   1.5189

[385 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
10        7  0.945310  actually
12        1  0.969319     added
31        8  0.922298   ancient
34        3  0.905856    animal
44        9  0.925665   appears
...     ...       ...       ...
1396      5  0.980947      year
1397      7  0.961049     years
1398      4  0.962810       yes
1401      1  0.986726   youtube
1402      6  0.963293      zone

[445 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 3, 10, 2, 7, 4, 9, 5, 1, 6])

##### Visualizing LDA results of a real disaster tweets with pyLDAvis

In [ ]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda_real, real_vectorized, vectorizer_real, mds='tsne')
dash

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
2      32.420952   8.115230       1        1  13.128126
0       0.648955  71.707100       2        1  11.566724
6      71.019684   9.971006       3        1  11.192658
8       3.785981  33.185135       4        1  10.914337
9     -42.666500  -9.222070       5        1  10.193261
3      -3.621689 -43.526516       6        1   9.802416
1      -3.632402  -4.171708       7        1   9.151980
4     -36.584400  35.804413       8        1   9.102390
5      44.391380  50.615383       9        1   7.826940
7      40.235809 -31.591665      10        1   7.121168, topic_info=          Term        Freq       Total Category  logprob  loglift
636   malaysia  185.000000  185.000000  Default  30.0000  30.0000
390     flight  126.000000  126.000000  Default  29.0000  29.0000
25    airlines  119.000000  119.000000  Default  28.0000  28.0000
1027   suicide  127.000000  127.000000  Default  27.0000  27.0000
363   families  106.000000  106.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
1217   youtube   13.020914   17.586503  Topic10  -4.6246   2.3415
386       flag    8.860378   10.725643  Topic10  -5.0095   2.4511
895        rly   10.886845   17.458652  Topic10  -4.8036   2.1698
64    attacked   10.654115   17.380936  Topic10  -4.8252   2.1527
1144     video   12.260583   67.857232  Topic10  -4.6847   0.9311

[410 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2         2  0.988919       abc
4         5  0.801245  accident
4         7  0.147598  accident
4         9  0.042171  accident
13        4  0.973785  affected
...     ...       ...       ...
1211      8  0.994747      year
1212      2  0.978408     years
1216      2  0.933648     youth
1217      5  0.227447   youtube
1217     10  0.739203   youtube

[465 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 7, 9, 10, 4, 2, 5, 6, 8])

#### TFIDF Vectorization

In [ ]:
tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
tfidf_vec.fit_transform(train_df['text_cleaned'].values.tolist() + test_df['text_cleaned'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['text_cleaned'].values.tolist())
test_tfidf = tfidf_vec.transform(test_df['text_cleaned'].values.tolist())

In [ ]:
train_tfidf

<7613x139122 sparse matrix of type '<class 'numpy.float64'>'
	with 174644 stored elements in Compressed Sparse Row format>

#### Building Classification models

##### 1. Logistic Regression

In [ ]:
train_y = train_df.target_relabeled.values

def runModel(train_X, train_y, test_X, test_y, test_X2):
  model = linear_model.LogisticRegression(C=5., solver='sag')
  model.fit(train_X, train_y)
  pred_test_y = model.predict_proba(test_X)[:,1]
  pred_test_y2 = model.predict_proba(test_X2)[:,1]
  return pred_test_y, pred_test_y2, model

print("Building model.")

cv_scores = []
pred_full_test = 0
pred_train = np.zeros([train_df.shape[0]])
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=8)
for dev_index, val_index in kf.split(train_df):
  dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
  dev_y, val_y = train_y[dev_index], train_y[val_index]
  pred_val_y, pred_test_y, model = runModel(dev_X, dev_y, val_X, val_y, test_tfidf)
  pred_full_test = pred_full_test + pred_test_y
  pred_train[val_index] = pred_val_y
  cv_scores.append(metrics.log_loss(val_y, pred_val_y))
  break

Building model.


In [ ]:
from tqdm import tqdm

def threshold_search(y_true, y_proba):
  #reference: https://www.kaggle.com/hung96ad/pytorch-starter
  best_threshold = 0
  best_score = 0
  for threshold in tqdm([i * 0.001 for i in range(1000)]):
    score = metrics.f1_score(y_true=y_true, y_pred=y_proba > threshold)
    print(score)
    if score > best_score:
      best_threshold = threshold
      best_score = score
      search_result = {'threshold': best_threshold, 'f1': best_score}
      return search_result
      
search_result = threshold_search(val_y, pred_val_y)
search_result

  0%|          | 0/1000 [00:00<?, ?it/s]

0.6059496567505721


{'f1': 0.6059496567505721, 'threshold': 0.0}

In [ ]:
print("F1 score at threshold {0} is {1}".format(0.381, metrics.f1_score(val_y, (pred_val_y>0.381).astype(int))))
print("Precision at threshold {0} is {1}".format(0.381, metrics.precision_score(val_y, (pred_val_y>0.381).astype(int))))
print("recall score at threshold {0} is {1}".format(0.381, metrics.recall_score(val_y, (pred_val_y>0.381).astype(int))))

F1 score at threshold 0.381 is 0.7624720774385705
Precision at threshold 0.381 is 0.7551622418879056
recall score at threshold 0.381 is 0.7699248120300752


In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

def getClassifierObj(classifier, rs):
    '''Functio to get the classifier object
    '''
    if classifier == 'XGBoost':
        from xgboost import XGBClassifier
        classifier = XGBClassifier()
    
    elif classifier == 'LogisticRegression':
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = rs)
    
    elif classifier == 'KNN':
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    
    elif classifier == 'SVM':
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'linear', random_state = rs)
        
    elif classifier == 'Kernel SVM':
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = rs)
        
    elif classifier == 'NB':
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        
    elif classifier == 'DecisionTree':
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = rs)
        
    elif classifier == 'RandomForest':
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = rs)
        
    return classifier
            
def fitAndPredict(estimator,X_train,y_train,X_test):
    '''Function to fit and predict
    '''
    estimator.fit(X_train,y_train)
    predictions = estimator.predict(X_test)
    return predictions

def getModelAccuracy(y_test, y_pred):
    '''Function to get model accuracy'''
    from sklearn.metrics import accuracy_score
    return accuracy_score(y_test, y_pred)*100

def getFinalPredictions(classifiers, X_train,X_test, y_train, y_test):
    ''' Function to get classifier with highest accuracy on the data
    '''
    # classifiers = ['LogisticRegression','KNN', 'Kernel SVM','DecisionTree','RandomForest','XGBoost']
    # classifiers = ['RandomForest']
    accuracy = 0
    best_predictions = ''
    bestClassifierName = ''
    classifier = ''
    
    for classifierName in classifiers:
        print('Evaluation started for ', classifierName)
        classifier = getClassifierObj(classifierName,8)
        y_pred = fitAndPredict(classifier,X_train,y_train,X_test)
        
        classifierAccuracy = getModelAccuracy(y_test,y_pred)
        print('Average accuracy of {} is {:.2f}%'.format(classifierName,classifierAccuracy))
        if classifierAccuracy > accuracy :
            accuracy = classifierAccuracy
            bestClassifierName = classifierName
            best_predictions = y_pred
            bestClassifier = classifier
            
            
    print('Classifier with highest accuracy is {}'.format(bestClassifierName))
    return best_predictions, accuracy, classifier      

def classify(classifiers, X_train,X_test, y_train, y_test):
    final_pred, final_accuracy, classifier = getFinalPredictions(classifiers, X_train,X_test, y_train, y_test)        
    cm = confusion_matrix(y_test, final_pred)
    print(cm)
    return final_pred, final_accuracy, classifier

In [ ]:

import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

classifiers = ['LogisticRegression','KNN', 'Kernel SVM','DecisionTree','RandomForest','XGBoost']
# classifiers = ['RandomForest']


train_y = train_df.target_relabeled.values
X_train, X_test, y_train, y_test = train_test_split(train_tfidf, train_y, random_state=8, test_size = 0.2)
final_pred, final_accuracy, bestClassifier = classify(classifiers,X_train, X_test, y_train, y_test)

Evaluation started for  LogisticRegression
Average accuracy of LogisticRegression is 74.85%
Evaluation started for  KNN
Average accuracy of KNN is 75.71%
Evaluation started for  Kernel SVM
Average accuracy of Kernel SVM is 73.08%
Evaluation started for  DecisionTree
Average accuracy of DecisionTree is 73.01%
Evaluation started for  RandomForest
Average accuracy of RandomForest is 75.77%
Evaluation started for  XGBoost
Average accuracy of XGBoost is 70.78%
Classifier with highest accuracy is RandomForest
[[812  49]
 [320 342]]


In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

# classifiers = ['LogisticRegression','KNN', 'Kernel SVM','DecisionTree','RandomForest','XGBoost']
classifiers = ['RandomForest']

train_y = train_df.target_relabeled.values
X_train, X_test, y_train, y_test = train_test_split(train_tfidf, train_y, random_state=8, test_size = 0.2)
final_pred, final_accuracy, bestClassifier = classify(classifiers,X_train, X_test, y_train, y_test)

Evaluation started for  RandomForest
Average accuracy of RandomForest is 76.43%
Classifier with highest accuracy is RandomForest
[[818  43]
 [316 346]]


### Cross Validation of models

In [ ]:
print(list(range(100,500,50)))

[100, 150, 200, 250, 300, 350, 400, 450]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV
parameters = [{'n_estimators' : list(range(100,300,50))}]

grid_search = GridSearchCV(estimator = RandomForestClassifier(),
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 3,
                           n_jobs = -1)

grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print('Best Accuracy: {:.2f}%'.format(best_accuracy*100))
print('Best Parameters:', best_parameters)
# train_scoreNum, test_scoreNum = validation_curve(RandomForestClassifier(),
#                                 X = X_train, y = y_train, 
#                                 param_name = 'n_estimators', 
#                                 param_range = num_est, cv = 3)

Best Accuracy: 76.58%
Best Parameters: {'n_estimators': 150}


In [ ]:
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
classifier = SVC()

parameters = [{'C': [1, 10, 100], 'kernel': ['linear']},
              {'C': [1, 10, 100], 'kernel': ['rbf'], 'gamma': [0.1, 0.5, 0.9]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 3,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print('Best Accuracy: {:.2f}%'.format(best_accuracy*100))
print('Best Parameters:', best_parameters)


Best Accuracy: 79.59%
Best Parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}


In [ ]:
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()

parameters = [{'n_neighbors' : list(range(5,50,2)),
               'leaf_size': [30,35,40,45,50],
               'metric': ['minkowski', 'euclidean']
               }]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 3,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print('Best Accuracy: {:.2f}%'.format(best_accuracy*100))
print('Best Parameters:', best_parameters)

Best Accuracy: 77.54%
Best Parameters: {'leaf_size': 30, 'metric': 'minkowski', 'n_neighbors': 43}


In [ ]:
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

parameters = [{'C': [1, 10, 100,1000], 
               'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
               'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
               }]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 3,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print('Best Accuracy: {:.2f}%'.format(best_accuracy*100))
print('Best Parameters:', best_parameters)

Best Accuracy: 79.67%
Best Parameters: {'C': 100, 'penalty': 'l1', 'solver': 'saga'}


In [ ]:
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold
from datetime import datetime
from xgboost import XGBClassifier

def timer(start_time=None):
  if not start_time:
    start_time = datetime.now()
    return start_time
  elif start_time:
    thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
    tmin, tsec = divmod(temp_sec, 60)
    print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))


params = {'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        
        }

xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=-1, cv=skf.split(X_train,y_train), verbose=3, random_state=1001 )

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train, y_train)
timer(start_time) # timing ends here for "start_time" variable 

print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 26.0min finished
/usr/local/lib/python3.6/dist-packages/xgboost/sklearn.py:242: DeprecationWarning:

The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.

/usr/local/lib/python3.6/dist-packages/xgboost/sklearn.py:249: DeprecationWarning:

The silent parameter is deprecated.Please use verbosity instead.silent is depreated




 Time taken: 0 hours 28 minutes and 0.62 seconds.

 All results:
{'mean_fit_time': array([164.6867017 , 234.52644324, 235.00639017, 177.8218534 ,
       189.56150119]), 'std_fit_time': array([ 0.61276134,  0.45626824,  1.82746491,  2.45172681, 47.9650434 ]), 'mean_score_time': array([0.14637272, 0.1670723 , 0.18501226, 0.16493742, 0.13338113]), 'std_score_time': array([0.00312982, 0.00222792, 0.00682904, 0.00268828, 0.02753328]), 'param_subsample': masked_array(data=[1.0, 0.6, 0.8, 1.0, 0.8],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[5, 1, 5, 5, 1],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[3, 5, 5, 5, 4],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[5, 1.5, 1, 5, 1],
             mask=[

In [ ]:
# Applying k-Fold Cross Validation
train_y = train_df.target_relabeled.values
k_fold = model_selection.KFold(n_splits=10, shuffle=True, random_state=8)
accuracies = []
for train, val in k_fold.split(train_tfidf):
  dev_X, val_X = train_tfidf[train], train_tfidf[val]
  dev_y, val_y = train_y[train], train_y[val]
  final_pred, final_accuracy, classifier = classify(['RandomForest'],dev_X, val_X, dev_y, val_y)
  accuracies.append(final_accuracy)



Evaluation started for  RandomForest
Average accuracy of RandomForest is 76.25%
Classifier with highest accuracy is RandomForest
[[400  30]
 [151 181]]
Evaluation started for  RandomForest
Average accuracy of RandomForest is 75.33%
Classifier with highest accuracy is RandomForest
[[407  25]
 [163 167]]
Evaluation started for  RandomForest
Average accuracy of RandomForest is 77.82%
Classifier with highest accuracy is RandomForest
[[407  24]
 [145 186]]
Evaluation started for  RandomForest
Average accuracy of RandomForest is 78.98%
Classifier with highest accuracy is RandomForest
[[433  18]
 [142 168]]
Evaluation started for  RandomForest
Average accuracy of RandomForest is 73.72%
Classifier with highest accuracy is RandomForest
[[408  28]
 [172 153]]
Evaluation started for  RandomForest
Average accuracy of RandomForest is 77.79%
Classifier with highest accuracy is RandomForest
[[440  15]
 [154 152]]
Evaluation started for  RandomForest
Average accuracy of RandomForest is 77.27%
Classifi

In [ ]:
print(np.mean(accuracies))

76.6058611924495


Classifier with the highest frequency is KNN and Random Forest.

In [ ]:
# LogisticRegression
final_classifier = LogisticRegression(C =100, penalty ='l1', solver= 'saga')
final_classifier.fit(X_train,y_train)
final_pred = final_classifier.predict(test_tfidf)
sub_df['target'] = final_pred


In [ ]:

# KNN
final_knn_classifier = KNeighborsClassifier(leaf_size= 30, metric= 'minkowski', n_neighbors= 43)
final_knn_classifier.fit(X_train,y_train)
final_knn_pred = final_knn_classifier.predict(test_tfidf)
sub_df['target'] = final_knn_pred


In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

final_rf_classifier = RandomForestClassifier(n_estimators = 150, criterion = 'entropy', random_state = 8)
final_rf_classifier.fit(X_train,y_train)
final_rf_pred = final_rf_classifier.predict(test_tfidf)
sub_df['target'] = final_rf_pred


In [ ]:
# XGB

final_xgb_classifier = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=1.5,
              learning_rate=0.02, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=600, n_jobs=1,
              nthread=1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=0.6, verbosity=1)

final_xgb_classifier.fit(X_train,y_train)
final_xgb_pred = final_xgb_classifier.predict(test_tfidf)
sub_df['target'] = final_xgb_pred

/usr/local/lib/python3.6/dist-packages/xgboost/sklearn.py:242: DeprecationWarning:

The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.

/usr/local/lib/python3.6/dist-packages/xgboost/sklearn.py:249: DeprecationWarning:

The silent parameter is deprecated.Please use verbosity instead.silent is depreated



In [ ]:
import csv
sub_df.to_csv('submission_rf.csv')